<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/langchain-notebooks/duckduckgo_youtube_api_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -q openai langchain wikipedia langchain_community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.2 MB/s eta 0:00:00


In [3]:
from openai import OpenAI
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [7]:
from langchain.llms import OpenAI

llm = OpenAI(api_key=OPENAI_API_KEY)
response = llm.generate(["농담 하나 해 주세요."])

In [9]:
print(response)

generations=[[Generation(text='\n왜 고양이가 말을 못해요? 키보드를 못 타니까요!', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 31, 'total_tokens': 42, 'prompt_tokens': 11}, 'model_name': 'gpt-3.5-turbo-instruct'} run=[RunInfo(run_id=UUID('b6e2e4e1-ae82-4425-9a2a-5fa961198979'))]


In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", api_key=OPENAI_API_KEY)
response = llm.generate(["농담 하나 해 주세요."])

In [12]:
print(response)

generations=[[ChatGeneration(text='왜 바나나가 보트를 타고 갔을까요?\n\n평범하게 걸으면 너무 미끄러워서요!', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='왜 바나나가 보트를 타고 갔을까요?\n\n평범하게 걸으면 너무 미끄러워서요!', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 18, 'total_tokens': 63}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f17db0dd-c701-47de-8ec2-0155310bcdef-0', usage_metadata={'input_tokens': 18, 'output_tokens': 45, 'total_tokens': 63}))]] llm_output={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 18, 'total_tokens': 63}, 'model_name': 'gpt-4'} run=[RunInfo(run_id=UUID('f17db0dd-c701-47de-8ec2-0155310bcdef'))]


In [17]:
!pip install -q duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.1 MB/s eta 0:00:00


In [22]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.run("Obama's first name?")

"Obama's father, Barack Obama, Sr., was a teenage goatherd in rural Kenya, won a scholarship to study in the United States, and eventually became a senior economist in the Kenyan government.Obama's mother, S. Ann Dunham, grew up in Kansas, Texas, and Washington state before her family settled in Honolulu.In 1960 she and Barack Sr. met in a Russian language class at the University of Hawaii ... The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. As the head of the government of the United States, the president is arguably the most powerful government official in the world. The president is elected to a four-year term via an electoral college system. Sinc

In [23]:
!pip install -q youtube-search

In [29]:
from langchain_community.tools import YouTubeSearchTool
search = YouTubeSearchTool()
search.run("오늘의 날씨")

"['https://www.youtube.com/watch?v=1d0u0R2xrj0&pp=ygUQ7Jik64qY7J2YIOuCoOyUqA%3D%3D', 'https://www.youtube.com/watch?v=lGLkUTTR9Pw&pp=ygUQ7Jik64qY7J2YIOuCoOyUqA%3D%3D']"

In [30]:
!pip install -q youtube-transcript-api pytube faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 68.7 MB/s eta 0:00:00


In [37]:
from langchain.document_loaders import YoutubeLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from pprint import pprint

def ask_youtube(url, q, language='ko'):
  loader = YoutubeLoader.from_youtube_url(url, language='ko')
  documents = loader.load()

  embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
  db = FAISS.from_documents(documents, embeddings)
  retriever = db.as_retriever()

  llm = ChatOpenAI(model_name="gpt-4", api_key=OPENAI_API_KEY)

  qa = RetrievalQA.from_chain_type(
      llm=llm,
      # chain_type="stuff",
      retriever=retriever)

  return qa({"query": q})

In [38]:
url = "https://www.youtube.com/watch?v=lvU-9qQhEnQ"
query = "조이보이가 뭐야?"

pprint(ask_youtube(url, query))

{'query': '조이보이가 뭐야?',
 'result': '조이보이는 원피스 만화에서 등장하는 미스터리한 인물로, 800년 전의 공백기간 동안 활동했다고 알려져 있습니다. 그는 '
           '20인의 왕들, 즉 세계 정부에 맞서 싸운 사람이었으며, 그의 이야기는 그 이후로 세계에서 잊혀진 채로 전해져 '
           '왔습니다. 그러나 그의 존재는 푸딩이 소유한 로드 폰 글리프에 기록되어 있었고, 이를 통해 그의 존재가 밝혀졌습니다. '
           '그의 정확한 신분이나 역할은 아직 밝혀지지 않았습니다.'}
